In [1]:
from src.anydoor_refiners.attention import CrossAttentionBlock2d
import torch
import cv2
import numpy as np

torch.set_num_threads(2)

device = 'cuda:0'
dtype = torch.float16

In [2]:
cross_attention = CrossAttentionBlock2d(
    channels=1280,
    context_embedding_dim=1024,
    context_key="dinov2_object_embedding",
    num_attention_heads=5,
    num_attention_layers=1,
    num_groups=32,
    use_bias=False,
    use_linear_projection=True,
    use_attention_tv_loss=True,
    device=device,
    dtype=dtype) 

In [3]:
mask = torch.zeros((1, 1, 512, 512), device=device, dtype=dtype)
box_x = (100, 200)
box_y = (100, 200)
mask[:, :, box_x[0]:box_x[1], box_y[0]:box_y[1]] = 1
latents = torch.randn(1, 1280, 16, 16, device=device, dtype=dtype)

embedding = torch.randn(1, 257, 1024, device=device, dtype=dtype)

In [4]:

cross_attention.set_context("atv_loss", {"value": torch.tensor(0.0, device=device, dtype=dtype)})
cross_attention.set_context("mask", {"mask": mask})
cross_attention.set_context("cross_attention_block", {"dinov2_object_embedding":embedding})

In [5]:
with torch.no_grad():
    output = cross_attention(latents)

256
I'm in
torch.Size([5, 256, 256])
257


/home/daniel/code/anydoor-refiners/.venv/lib/python3.11/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
cross_attention.use_context("atv_loss")["value"].item()

0.00018934524268843234

In [3]:
cross_attention

(RES) CrossAttentionBlock2d(channels=1280, context_embedding_dim=1024, num_attention_heads=5, use_bias=False, context_key=dinov2_object_embedding, use_attention_tv_loss=True, use_linear_projection=True)
    ├── (CHAIN) #1
    │   ├── GroupNorm(num_groups=32, eps=1e-06, channels=1280, device=cuda:0, dtype=float16)
    │   ├── (CHAIN) StatefulFlatten(start_dim=2)
    │   │   ├── SetContext(context=flatten, key=sizes)
    │   │   └── Flatten(start_dim=2)
    │   ├── Transpose(dim0=1, dim1=2)
    │   ├── Lambda(<lambda>(x))
    │   └── Linear(in_features=1280, out_features=1280, device=cuda:0, dtype=float16)
    ├── (CHAIN) #2
    │   └── (CHAIN) CrossAttentionBlock(embedding_dim=1280, context_embedding_dim=1024, context_key=dinov2_object_embedding, num_heads=5, use_bias=False)
    │       ├── (RES) Residual() #1
    │       │   ├── LayerNorm(normalized_shape=(1280,), device=cuda:0, dtype=float16)
    │       │   └── (CHAIN) SelfAttention(embedding_dim=1280, num_heads=5, inner_dim=1280, us